In [2]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from modules.backtest import cumplot
from pandas_datareader import DataReader as pdr

In [3]:
with create_engine("sqlite:///files/mydata.db").connect() as conn:
    rets = pd.read_sql("select * from model1", conn)
    rets["market"] = pd.read_sql("select market from market", conn)
rets = rets.set_index("date")

In [37]:
mns = 12*rets.mean()
sds = np.sqrt(12)*rets.std()
cov = 12*rets.cov()
rf = pdr("DGS1MO", "fred").iloc[-1].item() / 100
rprem = mns - rf
w = np.linalg.solve(cov, rprem)
w = w / np.sum(w)
mn = w @ mns
sd = np.sqrt(w @ cov @ w)
mxmn = 1.25*np.max((mns[0], mns[1], mns[2], mn))
mxsd = np.max((sds[0], sds[1], sds[2], sd))

w1 = np.linalg.solve(cov, np.ones(3))
w1 = w1 / np.sum(w1)
mn1 = w1 @ mns
w2 = np.linalg.solve(cov, mns)
w2 = w2 / np.sum(w2)
mn2 = w2 @ mns
def port(m):
    a = (m-mn2) / (mn1-mn2)
    return a*w1 + (1-a)*w2


In [40]:
trace1 = go.Scatter(
    x=sds,
    y=mns,
    text=rets.columns.to_list(),
    mode="markers",
    marker=dict(size=10),
    hovertemplate="%{text}<br>mn=%{y:.1%}<br>sd=%{x:.1%}<extra></extra>",
    showlegend=False
)

cd = np.empty(shape=(1, 3, 1), dtype=float)
cd[:, 0] = np.array(w[0])
cd[:, 1] = np.array(w[1])
cd[:, 2] = np.array(w[2])
string = "Tangency portfolio:<br>"
string += "best: %{customdata[0]:.1%}<br>"
string += "worst: %{customdata[1]:.1%}<br>"
string += "market: %{customdata[2]:.1%}<br>"
string += "<extra></extra>"
trace2 = go.Scatter(
    x=[sd],
    y=[mn],
    mode="markers",
    marker=dict(size=10),
    customdata=cd,
    hovertemplate=string,
    showlegend=False
)

x = np.linspace(0, mxsd, 51)
y = rf+x*(mn-rf)/sd
trace3 = go.Scatter(
        x=x,
        y=y,
        mode="lines",
        hovertemplate=f"Sharpe ratio = {(mn-rf)/sd:0.1%}<extra></extra>",
        showlegend=False,
    )

maxmn = np.max(y)
ms = np.linspace(np.min(mns), maxmn, 51)
ps = [port(m) for m in ms]
ss = [np.sqrt(p@cov@p) for p in ps]

def custom(string, ports):
    cd = np.empty(shape=(len(ports), 3, 1), dtype=float)
    for i in range(3):
        cd[:, i] = np.array([w[i] for w in ports]).reshape(-1, 1)
    string += "<br>"
    for i in range(3):
        string += "asset " + str(i + 1)
        string += ": %{customdata["
        string += str(i)
        string += "]:.0%}<br>"
    string += "<extra></extra>"
    return string, cd

string, cd = custom('frontier', ps)
trace4 = go.Scatter(
    x=ss,
    y=ms,
    mode="lines",
    customdata=cd,
    hovertemplate=string,
    showlegend=False,
    )

fig = go.Figure()
for trace in [trace1, trace2, trace3, trace4]:
    fig.add_trace(trace)
fig.update_layout(
        template="none",
        yaxis_tickformat=".0%",
        xaxis_tickformat=".0%",
        yaxis_title="Annualized Mean",
        xaxis_title="Annualized Standard Deviation",
        xaxis_rangemode="tozero",
        yaxis_rangemode="tozero",
    )
fig.show()